# 📘 Company CSR Question-Answer Retriever – V2

## 🔍 Objective

This notebook implements a Retrieval-Augmented Generation (RAG) pipeline for extracting information from Corporate Social Responsibility (CSR) reports. Users can ask natural language questions and receive grounded, context-aware answers based on the content of the uploaded PDF.

---

## 🧠 Pipeline Overview

The notebook follows a modular RAG architecture built using LangChain and OpenAI, structured as follows:

---

### 1. 📄 Document Loading

CSR reports in PDF format are parsed using multiple tools to handle structured and unstructured content:

- `pdfplumber` and `fitz` for text extraction.
- `pytesseract` and `PIL` for OCR of scanned images.
- `camelot` for extracting tabular data.

---

### 2. ✂️ Text Chunking

The extracted document is split into manageable text chunks using:

```python
from langchain.text_splitter import RecursiveCharacterTextSplitter
```

- Chunk size: 1000 characters  
- Overlap: 200 characters  

This ensures important context is preserved across chunks for better retrieval relevance.

---

### 3. 🧠 Embedding and Vector Storage

The chunked text is embedded and stored for semantic search using:

```python
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
```

- Each text chunk is embedded using OpenAI’s embedding model.
- The resulting vectors are stored in a Chroma vector store to enable efficient similarity-based retrieval.

---

### 4. 🔎 Query + Retrieval

When a user submits a question, the system performs semantic search to retrieve the most relevant chunks from the stored embeddings:

```python
docs = vectorstore.similarity_search(query)
```

These retrieved documents provide the contextual foundation for answer generation.

---

### 5. 🧾 Prompt Creation

A structured prompt is constructed by combining the retrieved context and the user’s question:

```python
from langchain.prompts import ChatPromptTemplate
```

Prompt format:

```
Use the following context to answer the question. Don’t guess.

{context}

Question: {question}
```

This ensures that the model generates responses grounded only in the available information.

---

### 6. 🤖 Response Generation

The prompt is passed to a language model to generate the final response:

```python
from langchain_openai import ChatOpenAI
```

The model uses the structured context and query to return a relevant and accurate answer.

---

## ✅ Summary

This notebook demonstrates a modular Retrieval-Augmented Generation (RAG) pipeline for answering natural language questions over CSR PDFs. It uses:

- Multi-tool PDF parsing  
- Overlapping text chunking  
- OpenAI-powered embeddings  
- Chroma-based semantic search  
- Prompt-based interaction with a chat model  

This setup is suitable for tasks such as ESG reporting, compliance auditing, and document intelligence across long, unstructured enterprise PDFs.


In [1]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import loads,dumps
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_unstructured import UnstructuredLoader
from langchain.docstore.document import Document
import fitz
import pytesseract
from PIL import Image
import io
import camelot
import pdfplumber
import time

In [2]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

In [3]:
load_dotenv(override = True)
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY','your-key-if-not-using-env')

In [4]:
def pdf_extractor(pdf_path, chunk_size=1000, chunk_overlap=200):
    print("Extracting info from",pdf_path) 
    doc = fitz.open(pdf_path)
    documents = []
    try:
        all_tables = camelot.read_pdf(pdf_path, pages='all')
        tables_by_page = {}
        for table in all_tables:
            page_num = table.parsing_report['page']
            if page_num not in tables_by_page:
                tables_by_page[page_num] = []
            tables_by_page[page_num].append(table)
    except:
        tables_by_page = {}
    
    for page_num in range(doc.page_count):
        page = doc[page_num]
        page_text = ""
        text = page.get_text()
        if text.strip():
            page_text += text
        current_page_num = page_num + 1
        table_count = 0
        
        if current_page_num in tables_by_page:
            for table in tables_by_page[current_page_num]:
                page_text += f"\n\n--- Table {table_count + 1} ---\n"
                page_text += table.df.to_string(index=False)
                table_count += 1
        else:
            try:
                with pdfplumber.open(pdf_path) as pdf_plumber:
                    if page_num < len(pdf_plumber.pages):
                        plumber_page = pdf_plumber.pages[page_num]
                        tables = plumber_page.extract_tables()
                        for table in tables:
                            if table and len(table) > 0:
                                page_text += f"\n\n--- Table {table_count + 1} ---\n"
                                if len(table) > 1:
                                    df = pd.DataFrame(table[1:], columns=table[0])
                                else:
                                    df = pd.DataFrame(table)
                                page_text += df.to_string(index=False)
                                table_count += 1
            except:
                pass
    
        image_list = page.get_images()
        image_count = 0
        
        for img_index, img in enumerate(image_list):
            try:
                xref = img[0]
                pix = fitz.Pixmap(doc, xref)
                if pix.n - pix.alpha < 4:
                    img_data = pix.tobytes("png")
                    pil_img = Image.open(io.BytesIO(img_data))
                    ocr_text = pytesseract.image_to_string(pil_img)
                    if ocr_text.strip():
                        page_text += f"\n\n[Image {image_count + 1} OCR]:\n{ocr_text}"
                        image_count += 1
                pix = None
            except:
                continue
        
        if page_text.strip():
            document = Document(
                page_content=page_text,
                metadata={
                    "source": pdf_path,
                    "page_number": page_num + 1,
                    "total_pages": doc.page_count,
                    "table_count": table_count,
                    "image_count": image_count,
                    "has_tables": table_count > 0,
                    "has_images": image_count > 0,
                    "content_length": len(page_text)
                }
            )
            documents.append(document)
    
    doc.close()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=[
            "\n\n--- Table",
            "\n\n[Image",
            "\n\n\n",
            "\n\n",
            "\n",
            ". ",
            " ",
            ""
        ]
    )
    
    chunks = text_splitter.split_documents(documents)
    
    for i, chunk in enumerate(chunks):
        chunk.metadata["chunk_id"] = i
        chunk.metadata["chunk_length"] = len(chunk.page_content)
    
    return chunks

In [5]:
def create_vector_db(db_name,embeddings,documents):
    if os.path.exists(db_name):
        print("Database exists.Proceeding to delete")
        Chroma(persist_directory = db_name, embedding_function = embeddings).delete_collection()
    vector_db = Chroma.from_documents(documents = documents, embedding = embeddings, persist_directory = db_name)
    retriever = vector_db.as_retriever()
    return retriever

In [6]:
def reciprocal_rank_fusion(results: list[list], k = 60):
    fused_scores = {}
    for list_doc in results:
        for rank,doc in enumerate(list_doc):
            #print("Document\n",doc)
            doc_str = dumps(doc)
            #print("Document\n",doc_str)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1/(rank+k)
            
    ranked_res = [(loads(doc),score) for doc,score in sorted(fused_scores.items(), key = lambda x: x[1], reverse = True)]
    
    return ranked_res

In [7]:
folder_path = "./Input Dataset/"
file_list = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

print(file_list)

['D4G_0.pdf', 'D4G_1.pdf']


In [8]:
db_name = 'vector_db'
embeddings = OpenAIEmbeddings()

In [9]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""

In [10]:
prompt_fus = ChatPromptTemplate.from_template(template)

In [11]:
generate_queries = (prompt_fus | ChatOpenAI(temperature = 0) | StrOutputParser() | (lambda x: x.split('\n')))

In [12]:
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

In [13]:
prompt = ChatPromptTemplate.from_template(template)

In [14]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [15]:
df = pd.read_csv('Output Format.csv')

In [16]:
queries = df.columns.to_list()[1:-1]

In [17]:
df.columns

Index(['Report Number ', 'Topic of Report ',
       'Name of Organisation who has generated the report',
       'Top 5 key highlights of the report like vision, mission etc.,s (In 200 words)',
       'Is this a new area that company is rolling out a CSR/ESG activity (Yes/ no)',
       'What is the stragey & Target of the the initaitive',
       'What is the spend forecast for the initiative ( numeric column )',
       'Furtue areas of investments based on the report (if any) in 200 words',
       'Previous areas that the company has worked in (in 200 words) if any ',
       'What locations are the initiatives plan (in 200 words)',
       'Max Time taken to run the entire code for each report'],
      dtype='object')

In [20]:
report_count = 0
final_df = pd.DataFrame()
for file in file_list:
    start = time.time()
    input_path = folder_path+file
    print(input_path)
    report_count +=1
    qna = {}
    qna['Report Number'] = report_count
    chunks = pdf_extractor(input_path, chunk_size=1200, chunk_overlap=150)
    retriever = create_vector_db(db_name,embeddings,documents = chunks)
    for question in queries:
        question += "?"
        print("Question Asked\n",question)
        retrieval_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
        final_chain = ({"context" : retrieval_fusion,
                        "question" : itemgetter("question")}
                       | prompt
                       | llm
                       | StrOutputParser()
                      )
        answer = final_chain.invoke({"question":question})
        qna[question] = answer
        print(answer)
    end = time.time()
    qna['Max Time'] = end-start
    data = pd.DataFrame([qna])
    final_df = pd.concat([final_df,data],ignore_index = True)

./Input Dataset/D4G_0.pdf
Extracting info from ./Input Dataset/D4G_0.pdf
Database exists.Proceeding to delete
Question Asked
 Topic of Report ?
The topic of the report is "Corporate Social Responsibility."
Question Asked
 Name of Organisation who has generated the report?
The name of the organization that generated the report is not explicitly mentioned in the provided context. The context only refers to the document as the "2021 CORPORATE SOCIAL RESPONSIBILITY REPORT" without specifying the organization's name.
Question Asked
 Top 5 key highlights of the report like vision, mission etc.,s (In 200 words)?
1. **Commitment to Sustainability**: Titan emphasizes its dedication to reducing environmental impact through eco-design principles and sustainable practices across its operations, including product design and manufacturing processes.

2. **Community Engagement**: The company recognizes its role in the communities it operates in, actively participating in local projects and encouragin

In [21]:
final_df

,Report Number,Topic of Report ?,Name of Organisation who has generated the report?,"Top 5 key highlights of the report like vision, mission etc.,s (In 200 words)?",Is this a new area that company is rolling out a CSR/ESG activity (Yes/ no)?,What is the stragey & Target of the the initaitive?,What is the spend forecast for the initiative ( numeric column )?,Furtue areas of investments based on the report (if any) in 200 words?,Previous areas that the company has worked in (in 200 words) if any ?,What locations are the initiatives plan (in 200 words)?,Max Time
0,1,"The topic of the report is ""Corporate Social R...",The name of the organization that generated th...,1. **Commitment to Sustainability**: Titan emp...,No.,The strategy of the initiative focuses on sust...,The provided context does not contain any spec...,The 2021 Corporate Social Responsibility Repor...,Titan has over 130 years of experience in the ...,The initiatives outlined in the document are p...,72.692199
1,2,"The topic of the report is ""Environmental, Soc...",The organization that generated the report is ...,The report emphasizes Broadcom's commitment to...,No.,The strategy of the initiative focuses on disc...,The provided context does not contain any info...,The report identifies several future areas of ...,"The company has focused on several key areas, ...",The initiatives mentioned in the context are p...,114.500301


In [ ]:
final_df.to_csv('Output.csv')